<a href="https://colab.research.google.com/github/LuchoValr/Machine_Learning/blob/main/Proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importacion de librerias

In [126]:
import pandas as pd

##Carga de datos

In [127]:
data=pd.read_csv('https://raw.githubusercontent.com/labeconometria/MLxE/main/proyectos1er/dataset_1.csv')
data.head()

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02


##Exploración inicial

In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1174 non-null   int64  
 1   gender     1174 non-null   object 
 2   wage       930 non-null    float64
 3   purchased  1174 non-null   int64  
 4   birth_day  1174 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 46.0+ KB


In [129]:
data.describe()

,user_id,wage,purchased
count,1174.000000,930.000000,1174.000000
mean,499.988927,71948.387097,0.404600
std,287.648865,34112.616989,0.491024
min,1.000000,15000.000000,0.000000
25%,252.250000,45500.000000,0.000000
50%,499.500000,71250.000000,0.000000
75%,749.750000,89500.000000,1.000000
max,1000.000000,152500.000000,1.000000


##Revisión de duplicados

In [130]:
data['user_id'].duplicated().sum()

174

In [131]:
data = data.drop_duplicates().reset_index(drop=True)
data

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02
...,...,...,...,...,...
995,305,Female,22500.0,1,1969-03-09
996,636,Male,76500.0,0,1970-08-26
997,320,Female,143500.0,1,1970-04-07
998,721,Male,148000.0,1,1993-11-23


In [132]:
data['gender'].value_counts()

Female    446
Male      423
woman      76
man        55
Name: gender, dtype: int64

In [133]:
data['gender'] = data['gender'].replace({'woman':'Female', 'man':'Male'})
data.head()

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02


In [134]:
data['gender'].value_counts()

Female    522
Male      478
Name: gender, dtype: int64

##Transformación de tipos de datos

In [135]:
from numpy import int32
data=data.astype({'user_id':'int32','purchased':'int32','wage':'float32'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       786 non-null    float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
dtypes: float32(1), int32(2), object(2)
memory usage: 27.5+ KB


##Enriquecimiento de datos

In [136]:
age = data['birth_day'].str.split('-', expand=True)
age = age[0].astype('int32')
age

0      1983
1      1972
2      1981
3      1983
4      1967
       ... 
995    1969
996    1970
997    1970
998    1993
999    1984
Name: 0, Length: 1000, dtype: int32

In [137]:
data['age'] = 2023 - age
data.head()


,user_id,gender,wage,purchased,birth_day,age
0,217,Female,NaN,0,1983-08-06,40
1,226,Female,146500.0,1,1972-01-28,51
2,697,Male,NaN,1,1981-12-12,42
3,743,Female,NaN,0,1983-05-26,40
4,63,Female,140500.0,1,1967-11-02,56


##Missing values

In [138]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       786 non-null    float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
 5   age        1000 non-null   int32  
dtypes: float32(1), int32(3), object(2)
memory usage: 31.4+ KB


In [139]:
data['wage'].median()

71750.0

In [140]:
data.pivot_table(index=['gender','age'],values='wage',aggfunc=['mean','median'])

mean    median
                     wage      wage
gender age                         
Female 19    77000.000000   77000.0
       20    55875.000000   57500.0
       21    47400.000000   27500.0
       22    63000.000000   68000.0
       23    42250.000000   41000.0
...                   ...       ...
Male   60   121625.000000  139250.0
       61    72857.140625   62500.0
       62    74500.000000   43500.0
       63   139000.000000  139000.0
       64    95000.000000   95000.0

[92 rows x 2 columns]

In [141]:
data['wage'] = data['wage'].fillna(data.groupby(['gender','age'])['wage'].transform('median'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       1000 non-null   float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
 5   age        1000 non-null   int32  
dtypes: float32(1), int32(3), object(2)
memory usage: 31.4+ KB


##Outliers

In [ ]:
q1 = data["wage"].quantile(0.25)
q3 = data["wage"].quantile(0.75)
IQR = q3 - q1
lim_sup = q3 + (1.5 * IQR)
lim_inf = q1 - (1.5 * IQR)

In [ ]:
lim_inf,lim_sup

(-4437.5, 142062.5)

##Datos inconsistentes

##Manejo de memoria

##Clasificación de datos

In [ ]:
data['cat_age'] = pd.cut(data.age, bins=[0, 25, 30, 60, float('Inf')], labels=['teen', 'young','adult', 'old'])
data.head()

,user_id,gender,wage,purchased,birth_day,age,cat_age
0,217,Female,66500.0,0,1983-08-06,40,adult
1,226,Female,146500.0,1,1972-01-28,51,adult
2,697,Male,66500.0,1,1981-12-12,42,adult
3,743,Female,66500.0,0,1983-05-26,40,adult
4,63,Female,140500.0,1,1967-11-02,56,adult


In [ ]:
data['wage'].max()

152500.0

In [ ]:
data['cat_wage'] = pd.cut(data.wage, bins=[0, 5000, 10000, 50000, 100000, float('Inf')], labels=['low', 'low-medium', 'medium', 'medium-high', 'high'])
data.head()

,user_id,gender,wage,purchased,birth_day,age,cat_age,cat_wage
0,217,Female,66500.0,0,1983-08-06,40,adult,medium-high
1,226,Female,146500.0,1,1972-01-28,51,adult,high
2,697,Male,66500.0,1,1981-12-12,42,adult,medium-high
3,743,Female,66500.0,0,1983-05-26,40,adult,medium-high
4,63,Female,140500.0,1,1967-11-02,56,adult,high


##Porcentajes

In [ ]:
table = data.pivot_table('purchased', 'cat_age')
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

,purchased,% purchased
cat_age,,
teen,0.000000,0.000000
young,0.061947,4.252497
adult,0.461438,31.676540
old,0.933333,64.070962


In [ ]:
table = data.pivot_table('purchased', 'cat_wage')
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

,purchased,% purchased
cat_wage,,
medium,0.437751,28.192259
medium-high,0.248705,16.017202
high,0.866279,55.790538


In [ ]:
table = data.pivot_table('purchased', 'gender')
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

,purchased,% purchased
gender,,
Female,0.423372,52.787328
Male,0.378661,47.212672


In [ ]:
table = data.pivot_table('purchased', ['gender','cat_wage','cat_age'])
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

purchased  % purchased
gender cat_wage    cat_age                        
Female medium      teen      0.000000     0.000000
                   young     0.000000     0.000000
                   adult     0.593407     5.647003
                   old       0.846154     8.052209
       medium-high teen      0.000000     0.000000
                   young     0.081081     0.771588
                   adult     0.274038     2.607818
                   old       0.833333     7.930205
       high        young     0.333333     3.172082
                   adult     0.834951     7.945604
                   old       1.000000     9.516247
Male   medium      teen      0.000000     0.000000
                   young     0.000000     0.000000
                   adult     0.486842     4.632910
                   old       1.000000     9.516247
       medium-high teen      0.000000     0.000000
                   young     0.000000     0.000000
                   adult     0.302128     2.875121
                   old       1.000000     9.516247
       high        young     1.000000     9.516247
                   adult     0.923077     8.784228
                   old       1.000000     9.516247

In [ ]:
data.pivot_table('wage','cat_wage',aggfunc=['mean','median'])

,mean,median
,wage,wage
cat_wage,,
medium,34538.152344,34500.0
medium-high,72188.687500,73000.0
high,125687.500000,124750.0


In [ ]:
data.pivot_table('wage','gender',aggfunc=['mean','median'])

,mean,median
,wage,wage
gender,,
Female,73376.437500,71250.0
Male,70529.289062,71000.0
